# 🔌CONEXÃO
Estabelecer conexão com os bancos

In [2]:
import pyodbc
import fdb
from dotenv import load_dotenv
import os

load_dotenv()

# Conexões
cnx_dest = fdb.connect(
    user=os.getenv("FDB_USER"),
    password=os.getenv("FDB_PASS"),
    host=os.getenv("FDB_HOST"),
    port=int(os.getenv("FDB_PORT")),
    database=os.getenv("FDB_PATH"),
    charset="WIN1252"
)
cur_dest = cnx_dest.cursor()


cnx_orig = pyodbc.connect(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={os.environ['SQLS_HOST']},1433;"
    f"DATABASE={os.environ['SQLS_DB']};"
    f"UID={os.environ['SQLS_USER']};"
    f"PWD={os.environ['SQLS_PASS']};"
    "TrustServerCertificate=Yes;"
)
cnx_orig.autocommit = True
cur_orig = cnx_orig.cursor()

def commit():
    cnx_dest.commit()

# 🛠️ FERRAMENTAS
Funções, variáveis cache, hashmaps

In [3]:
global cadest, empresa, exercicio
cadest = {}
empresa = cur_dest.execute("SELECT empresa FROM cadcli").fetchone()[0]
exercicio = f'{os.getenv("ANO")}'

def limpa_tabela(tabelas):
    for tabela in tabelas:
        cur_dest.execute(f"DELETE FROM {tabela}")
    commit()

def cria_coluna(tabela, coluna):
    try:
        cur_dest.execute(f"ALTER TABLE {tabela} ADD {coluna} VARCHAR(255)")
    except fdb.DatabaseError as e:
        print(f"Erro ao criar coluna {coluna} na tabela {tabela}: {e}")
    else:
        commit()

dict_modalidades = {
"1.1":{"Licit": "MAT / SERV - CONCORRENCIA", 
      "Modlic": "CON4",
      "Codmod": 4},
"2.1":{"Licit": "MAT / SERV - TOMADA", 
      "Modlic": "TOM3",
      "Codmod": 3},
"3.1":{"Licit": "MAT / SERV - CONVITE", 
      "Modlic": "CS01",
      "Codmod": 7},
"14.1":{"Licit": "DISPENSA", 
      "Modlic": "DI01",
      "Codmod": 1},
"8.1":{"Licit": "INEXIGIBILIDADE", 
      "Modlic": "IN01",
      "Codmod": 5},
"6.1":{"Licit": "PREGÃO PRESENCIAL", 
      "Modlic": "PP01",
      "Codmod": 8},
"6.2":{"Licit": "PREGÃO ELETRÔNICO", 
      "Modlic": "PE01",
      "Codmod": 9},
"9.1":{"Licit": "DISPENSA", 
      "Modlic": "DI01",
      "Codmod": 1},
"7.1":{"Licit": "DISPENSA", 
      "Modlic": "DI01",
      "Codmod": 1},
"13.1":{"Licit": "DISPENSA", 
      "Modlic": "DI01",
      "Codmod": 1},
"11.1":{"Licit": "DISPENSA", 
      "Modlic": "DI01",
      "Codmod": 1},
}

def cria_coluna(tabela, coluna):
    try:
        cur_dest.execute(f"ALTER TABLE {tabela} ADD {coluna} VARCHAR(255)")
    except fdb.DatabaseError as e:
        print(f"Erro ao criar coluna {coluna} na tabela {tabela}: {e}")
    else:
        commit()

def to_cp1252_safe(value, replace_with=''):
    """
    Converte uma string para cp1252 de forma segura.
    Remove ou substitui caracteres inválidos.

    :param value: valor a converter
    :param replace_with: string usada no lugar de caracteres inválidos (default = '')
    :return: valor limpo
    """
    if isinstance(value, str):
        try:
            # Tenta converter diretamente
            value.encode('cp1252')
            return value
        except UnicodeEncodeError:
            # Remove/substitui caracteres inválidos
            return value.encode('cp1252', errors='replace').decode('cp1252').replace('?', replace_with)
    return value

cur_dest.execute("SELECT codreduz, max(cadpro) FROM cadest group by 1")
if cur_dest.description is None:
    print("CADEST VAZIA!")
else:
    cadest = {k:v for k,v in cur_dest.execute("SELECT codreduz, max(cadpro) FROM cadest group by 1").fetchall()}

try:
    unidades = {k: v for k, v in cur_dest.execute("SELECT codant, sigla FROM cadunimedida").fetchall()}
except:
    pass

fornecedores = {k:v for k,v in cur_dest.execute("SELECT id_fornecedor_ant, codif FROM desfor").fetchall()}


# 🛒 COMPRAS
<p>Extração, tratamento e carregamento dos dados referentes ao módulo compras</p>

In [4]:
cur_dest.execute("""
execute block as
		begin
		DELETE FROM ICADREQ;
		DELETE FROM REQUI;
		DELETE FROM ICADPED;
		DELETE FROM CADPED;
		DELETE FROM regpreco;
		DELETE FROM regprecohis;
		DELETE FROM regprecodoc;
		DELETE FROM CADPRO_SALDO_ANT;
		DELETE FROM CADPROLIC_DETALHE_FIC;
		DELETE FROM CADPRO;
		DELETE FROM CADPRO_FINAL;
		DELETE FROM CADPRO_LANCE;
		DELETE FROM CADPRO_PROPOSTA;
		DELETE FROM PROLICS;
		DELETE FROM PROLIC;
		DELETE FROM CADPRO_STATUS;
		DELETE FROM CADLIC_SESSAO;
		DELETE FROM CADPROLIC_DETALHE;
		DELETE FROM CADPROLIC;
		DELETE FROM CADLIC;
		DELETE FROM VCADORC;
		DELETE FROM FCADORC;
		DELETE FROM ICADORC;
		DELETE FROM CADORC;
		DELETE FROM CADEST;
		DELETE FROM CENTROCUSTO;
		DELETE FROM DESTINO;
		DELETE FROM DESFORCRC_PADRAO;
		end;
""")
commit()

## CADASTROS BASE

### CADUNIMEDIDA

In [5]:
limpa_tabela(("cadunimedida",))
cria_coluna("cadunimedida", "codant")
cria_coluna("cadunimedida", "codtce")

insert = cur_dest.prep("""
    INSERT INTO cadunimedida(sigla, descricao, codant, codtce)
    VALUES (?, ?, ?, ?)
""")

cur_orig.execute("""
    SELECT
        substring(trim(sigla),1,5) AS sigla,
        substring(Descricao,1,30) AS descricao,
        id,
        CodigoTCEUnidFornecimento AS codtce
    FROM UnidMedidaEmbalagem
""")

dict_auxiliar_cadunidades = {}

for row in cur_orig:
    sigla_base = row.sigla.strip()
    contador = dict_auxiliar_cadunidades.get(sigla_base, 0)
    
    while True:
        if contador == 0:
            nova_sigla = sigla_base
        else:
            sufixo = str(contador)
            tamanho_prefixo = 5 - len(sufixo)
            nova_sigla = (sigla_base[:tamanho_prefixo] + sufixo)[:5]
        
        if nova_sigla not in dict_auxiliar_cadunidades:
            break
        contador += 1
    

    dict_auxiliar_cadunidades[sigla_base] = contador + 1
    dict_auxiliar_cadunidades[nova_sigla] = 1 

    cur_dest.execute(insert, (nova_sigla, row.descricao, row.id, row.codtce))
cur_dest.execute(insert, ('UNIDE', 'UNIDADE', 0, None))
commit()

unidades = {k: v for k, v in cur_dest.execute("SELECT codant, sigla FROM cadunimedida").fetchall()}

Erro ao criar coluna codant na tabela cadunimedida: ('Error while executing SQL statement:\n- SQLCODE: -607\n- unsuccessful metadata update\n- unknown ISC error 336397287\n- violation of PRIMARY or UNIQUE KEY constraint "RDB$INDEX_15" on table "RDB$RELATION_FIELDS"\n- unknown ISC error 335545072', -607, 335544351)
Erro ao criar coluna codtce na tabela cadunimedida: ('Error while executing SQL statement:\n- SQLCODE: -607\n- unsuccessful metadata update\n- unknown ISC error 336397287\n- violation of PRIMARY or UNIQUE KEY constraint "RDB$INDEX_15" on table "RDB$RELATION_FIELDS"\n- unknown ISC error 335545072', -607, 335544351)


### GRUPO E SUBGRUPO

In [6]:
limpa_tabela(("cadsubgr", "cadgrupo"))
cria_coluna("cadsubgr", "codant")
cria_coluna("cadsubgr", "codtce")

cur_orig.execute("""
with cte as (
select distinct
	concat(substring(format(id, '0000'),1,1),'0',abs(IdTipoItemBase)) grupo,
	'0000' subgrupo,
	case when abs(IdTipoItemBase) = 2 then 'SERVIÇOS' else 'PRODUTOS' END nome,
	NULL codant,
	NULL codtce
from
	itembase
union all
select distinct
	concat(substring(format(id, '0000'),1,1),'0',abs(IdTipoItemBase)) grupo,
	substring(format(id+1, '0000'),2,3) subgrupo,
	Descricao nome,
	id codant,
	CodigoTCE
from
	itembase
)
select * from cte
""")

for row in cur_orig:
	try:
		if row.subgrupo == '0000':
			cur_dest.execute("insert into cadgrupo(grupo, nome) values(?, ?)", (row.grupo, row.nome[:45]))
		else:
			cur_dest.execute("insert into cadsubgr(grupo, subgrupo, nome, codant, codtce) values(?, ?, ?, ?,? )", (row.grupo, row.subgrupo, row.nome[:45], row.codant, row.codtce))
		commit()
	except Exception as e:
		print(f"Erro ao inserir grupo/subgrupo {row.grupo}/{row.subgrupo}: {e}")


cur_dest.execute("insert into cadgrupo(grupo, nome) values ('000', 'SEM IDENTIFICAÇÃO')")
commit()

cur_dest.execute("insert into cadsubgr(grupo, subgrupo, nome) values ('000', '000', 'SEM IDENTIFICAÇÃO')")
commit()

Erro ao criar coluna codant na tabela cadsubgr: ('Error while executing SQL statement:\n- SQLCODE: -607\n- unsuccessful metadata update\n- unknown ISC error 336397287\n- violation of PRIMARY or UNIQUE KEY constraint "RDB$INDEX_15" on table "RDB$RELATION_FIELDS"\n- unknown ISC error 335545072', -607, 335544351)
Erro ao criar coluna codtce na tabela cadsubgr: ('Error while executing SQL statement:\n- SQLCODE: -607\n- unsuccessful metadata update\n- unknown ISC error 336397287\n- violation of PRIMARY or UNIQUE KEY constraint "RDB$INDEX_15" on table "RDB$RELATION_FIELDS"\n- unknown ISC error 335545072', -607, 335544351)


### CADEST

In [7]:
limpa_tabela(("cadest",))
cria_coluna("cadest", "key")

insert = cur_dest.prep("insert into cadest(grupo, subgrupo, codigo, cadpro, codreduz, disc1, ocultar, unid1, tipopro, usopro, key) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)")

cur_orig.execute("""
select
	b.grupo,
	b.subgrupo,
	a.Id codreduz,
	trim(a.Descricao) disc1,
	'N' ocultar,
	case when coalesce(c.IdUnidMedidaEmbalagem,0) <= 0 then 0 else c.IdUnidMedidaEmbalagem end codunid,
	case when grupo like '%002' then '3' when grupo like '4' then 2 else 1 end tipo
from
	Item a
join (select distinct
	concat(substring(format(id, '0000'),1,1),'0',abs(IdTipoItemBase)) grupo,
	substring(format(id+1, '0000'),2,3) subgrupo,
	Descricao nome,
	id codant,
	CodigoTCE
from
	itembase) b on a.IdItemBase = b.codant
LEFT JOIN (select Iditem, max(IdUnidMedidaEmbalagem) IdUnidMedidaEmbalagem from ItemXUnidMedidaEmbalagem group by IdItem) c on c.IdItem = a.Id 
""")

subgrupos_desdobrados = {k:[v, 0] for k, v in cur_dest.execute("select grupo||'.'||subgrupo, cast(subgrupo as integer) subgrupo_int from cadsubgr")}
max_subgrupo_grupo = {k: v for k, v in cur_dest.execute("select grupo, max(cast(subgrupo as integer)) from cadsubgr group by 1")}

tipo_uso_produto = {
	1: ['P', 'C'],
	2: ['P', 'P'],
	3: ['S', '']
}

for row in cur_orig.fetchall():
	key = f"{row.grupo}.{row.subgrupo}"
	subgrupo_int, codigo = subgrupos_desdobrados[key]
	codigo += 1

	if codigo > 999:
		subgrupo_ant = max_subgrupo_grupo[row.grupo]
		subgrupo_int = max_subgrupo_grupo[row.grupo] + 1
		codigo = 1
		subgrupos_desdobrados[key] = [subgrupo_int, codigo]
		max_subgrupo_grupo[row.grupo] = subgrupo_int
	else:
		subgrupos_desdobrados[key][1] = codigo

	tipopro, usopro = tipo_uso_produto.get(row.tipo, ['P', 'C'])

	cadpro = f'{row.grupo}.{subgrupo_int:03}.{codigo:03}'

	unid1 = unidades[str(row.codunid)]

	cur_dest.execute(insert, (row.grupo, f'{subgrupo_int:03}', f'{codigo:03}', cadpro, row.codreduz, row.disc1[:1024], 'N', unid1, tipopro, usopro, key))
commit()

cur_dest.execute("""
INSERT INTO CADSUBGR (grupo, subgrupo, nome)
WITH CTE AS (
SELECT DISTINCT a.grupo, a.subgrupo, KEY FROM cadest a
WHERE NOT EXISTS (SELECT 1 FROM cadsubgr x WHERE a.grupo = x.grupo AND a.SUBGRUPO = x.SUBGRUPO))
SELECT a.GRUPO, a.SUBGRUPO, b.NOME FROM cte a
JOIN CADSUBGR b ON b.grupo||'.'||b.SUBGRUPO = a."KEY" 
""")
commit()

cadest = {k: v for k,v in cur_dest.execute("select codreduz, max(cadpro) from cadest group by codreduz").fetchall()}

### ALMOXARIFADOS

In [8]:
limpa_tabela(("destino",))
cria_coluna("destino", "almox_pai")

insert = cur_dest.prep("""insert into destino(cod,desti,empresa,almox_pai) values(?,?,?,?)""")

cur_dest.execute(insert, ('000000000', 'CONVERSAO', empresa, '000000000'))
commit()

### CENTRO DE CUSTO

In [9]:
limpa_tabela(("centrocusto",))
cria_coluna("centrocusto", "pai")

try:
    cur_dest.execute(f"insert into destino(cod,desti,empresa) values('000000000','CONVERSAO',{empresa})")
except fdb.DatabaseError as e:
    print(f"Erro ao inserir destino: {e}")
else:
    commit()

insert = cur_dest.prep("""
    insert into centrocusto (codccusto, descr, ccusto, ocultar, empresa, poder, orgao, destino, pai) values (?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

cur_orig.execute("""
select id codccusto, descricao descr from dbo.estruturaHierarquica a where exists (select 1 from dbo.ItemOrdemForn iof where iof.IdEstrutAdministrativa = a.id)
""")

for row in cur_orig:
    cur_dest.execute(insert, (row.codccusto, row.descr, '001', 'N', empresa, '02', '01', '000000000', None))
cur_dest.execute(insert, (0, 'CONVERSÃO', '001', 'N', empresa, '02', '01', '000000000', None))
commit()

Erro ao inserir destino: ('Error while executing SQL statement:\n- SQLCODE: -803\n- attempt to store duplicate value (visible to active transactions) in unique index "XPKDESTINO"\n- unknown ISC error 335545072', -803, 335544349)




## COTAÇÃO



In [10]:
cur_dest.execute("""
execute block as
    begin
    DELETE FROM vcadorc;
    DELETE FROM fcadorc;
    DELETE FROM icadorc;
    DELETE FROM cadorc;
end;
""")
commit()

### CADORC

In [11]:
limpa_tabela(("cadorc",))

insert = cur_dest.prep("""
insert into cadorc
	(id_cadorc,
	num,
	ano,
	numorc,
	dtorc,
	descr,
	prioridade,
	obs,
	status,
	liberado,
	codccusto,
	liberado_tela,
	empresa,
    codif) values
	(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

cur_orig.execute("""
select
	a.id,
	numero,
	a.DataCotacao dtorc,
	a.Observacao descr,
	a.Observacao obs,
	case
		when IdSituacaoCotacao = -1 then 'AB'
		when IdSituacaoCotacao = -2 then 'LC'
		else 'CA'
	end status,
	IdFornecedorUG codif,
	a.IdUnidadeGestora codccusto,
	b.NomeRazaoSocial
from
	dbo.Cotacao a
left join (
	select
		a.Id,
		b.NomeRazaoSocial
	from
		dbo.FornecedorUG a
	join dbo.Pessoa b on
		a.IdPessoa = b.Id) b on
	a.IdFornecedorUG = b.Id
""")

for row in cur_orig:
	num, ano = row.numero.split('/')
	numorc = f'{num.zfill(5)}/{int(ano)%2000}'

	if row.status == 'LC':
		liberado = 'S'
		liberado_tela = 'L'
	else:
		liberado = 'N'
		liberado_tela = 'N'

	try:
		cur_dest.execute(insert, (
			row.id,
			num,
			ano,
			numorc,
			row.dtorc,
			row.descr[:1024],
			'NORMAL',
			row.obs,
			row.status,
			liberado,
			row.codccusto,
			liberado_tela,
			empresa,
			row.codif
		))
	except Exception as e:
		print(f"Error: {e} - id: {row.id}")
commit()

### ICADORC

In [ ]:
limpa_tabela(("icadorc",))

insert = cur_dest.prep("""
insert into icadorc (
    numorc,
    item,
    cadpro,
    qtd, 
    valor, 
    itemorc, 
    codccusto, 
    itemorc_ag,
    id_cadorc)
values(?,?,?,?,?,?,?,?,?)
""")

cur_orig.execute("""select
	b.Numero numero,
	a.Sequencia item,
	a.IdItem codreduz,
	a.Quantidade qtd,
	0 valor,
	IdUnidadeGestora codccusto,
	b.id id_cadorc
from
	dbo.ItemCotacao a
join dbo.Cotacao b on a.IdCotacao = b.id
""")

for row in cur_orig:
    try:
        key = str(row.codreduz)
        cadpro = cadest[key]
        num, ano = row.numero.split('/')
        numorc = f'{num.zfill(5)}/{int(ano)%2000}'
    except Exception as e:
        print(f"Erro ao obter cadpro: {e}")
        break

    else:
        cur_dest.execute(insert, (
            numorc,
            row.item,
            cadpro,
            row.qtd,
            row.valor,
            row.item,
            0,
            row.item,
            row.id_cadorc
        ))
commit()    

### FCADORC

In [ ]:
limpa_tabela(("fcadorc",))

insert = cur_dest.prep("insert into fcadorc(numorc, codif, nome, valorc, id_cadorc) values (?,?,?,?,?)")

cur_orig.execute("""
with CTE as (
select
	b.Numero,
	a.Sequencia item,
	a.IdItem,
	a.Quantidade qtd,
	a.ValorTotal,
	IdUnidadeGestora codccusto,
	b.id,
	codif,
	b.NomeRazaoSocial 
from
	dbo.ItemCotacao a
join (
select
	a.id,
	numero,
	a.DataCotacao,
	a.Observacao descr,
	'NORMAL' prioridade,
	a.Observacao obs,
	case
		when IdSituacaoCotacao = -1 then 'AB'
		when IdSituacaoCotacao = -2 then 'EC'
		else 'CA'
	end status,
	b.codif,
	a.IdUnidadeGestora,
	b.NomeRazaoSocial
from
	dbo.Cotacao a
left join (
	select
		a.Id,
		b.Id codif,
		b.NomeRazaoSocial
	from
		dbo.FornecedorUG a
	join dbo.Pessoa b on
		a.IdPessoa = b.Id) b on
	a.IdFornecedorUG = b.Id) b on a.IdCotacao = b.id) 
select numero, codif, NomeRazaoSocial nome, sum(ValorTotal) valorc, id id_cadorc from cte
group by numero, codif, NomeRazaoSocial, id
""")

for row in cur_orig:
	num, ano = row.numero.split('/')
	numorc = f'{num.zfill(5)}/{int(ano)%2000}'
	codif = fornecedores[row.codif]

	cur_dest.execute(insert, (
		numorc,
		codif,
		row.nome[:70],
		row.valorc,
		row.id_cadorc
	))
commit()

### VCADORC

In [ ]:
limpa_tabela(("vcadorc",))

insert = cur_dest.prep("""insert into vcadorc(numorc, item, codif, vlruni, vlrtot, ganhou, vlrganhou, classe, id_cadorc) values (?,?,?,?,?,?,?,?,?)""")

cur_orig.execute("""
select
	b.Numero numero,
	a.Sequencia item,
	c.idPessoa  codif,
	a.IdItem codreduz,
	a.Quantidade qtd,
	a.ValorUnitario vlruni,
	a.ValorTotal vlrtot,
	IdUnidadeGestora codccusto,
	b.id id_cadorc
from
	dbo.ItemCotacao a
join dbo.Cotacao b on a.IdCotacao = b.id
join (select id, idPessoa from dbo.FornecedorUG fu) c on b.IdFornecedorUG = c.id
order by 1, 2, 3
""")

for row in cur_orig:
    num, ano = row.numero.split('/')
    numorc = f'{num.zfill(5)}/{int(ano)%2000}'
    codif = fornecedores[row.codif]

    cur_dest.execute(insert, (
        numorc,
        row.item,
        codif,
        row.vlruni,
        row.vlrtot,
        row.codif,
        row.vlruni,
        'UN',
        row.id_cadorc
    ))
commit()

## LICITAÇÕES

In [ ]:
#LIMPA LICITAÇÕES
cur_dest.execute("""
execute block as
    begin
    DELETE FROM regpreco;
    DELETE FROM regprecohis;
    DELETE FROM regprecodoc;
    DELETE FROM CADPROLIC_DETALHE_FIC;
    DELETE FROM CADPRO;
    DELETE FROM CADPRO_FINAL;
    DELETE FROM CADPRO_LANCE;
    DELETE FROM CADPRO_PROPOSTA;
    DELETE FROM PROLICS;
    DELETE FROM PROLIC;
    DELETE FROM CADPRO_STATUS;
    DELETE FROM CADLIC_SESSAO;
    DELETE FROM CADPROLIC_DETALHE;
    DELETE FROM CADPROLIC;
    DELETE FROM CADLIC;
    end;
""")
commit()

### CADLIC

In [ ]:
limpa_tabela(("cadlic",))

insert = cur_dest.prep("""
INSERT
		INTO
		cadlic(
		numlic,
		proclic,
		numero,
		ano,
		comp,
		licnova,
		liberacompra,
		discr,
		detalhe,
		registropreco,
		microempresa,
		numpro,
		discr7,
		datae,
		processo_data,
		dtadj,
		dthom,
		codtce,
		anomod,
		modlic,
		licit,
		codmod,
		dtpub,
		dtenc,
		valor,
		empresa,
		processo,
		processo_ano,
		dtreal,
        numorc,
        id_cadorc,
		tpapostilamento,
		mascmod)
	VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
	?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
    ?, ?, 'I', ?)
""")

cur_orig.execute("""
select
	a.id numlic,
	concat(format(a.id , '000000/'), ano%2000) proclic,
	Ano ano,
	IdSituacaoLicitacao,
	1 comp,
	1 licnova,
	DescricaoObjeto  discr,
	b.JustificativaLicitacao  detalhe,
	case when FlGeraRegistroPrecos = 1 then 'S' else 'N' end registropreco,
	3 microempresa,
	Numero numpro,
	'Menor Preco Unitario' discr7,
	DataAbertura datae,
	DataProcessoAdm processo_data,
	DataHoraAdjudicacao dtadj,
	DataHoraHomologacao dthom,
	CodigoIdentificacaoTCE codtce,
	ano anomod,
	concat(abs(IdModalidadeLicitacao),'.',coalesce(abs(IdFormaRealizacaoLicitacao),1)) modlic,
	Null dtpub,
	DataHoraHomologacao dtenc,
	ValorEdital valor,
	NumeroProcAdm processo,
	AnoProcAdm processo_ano,
	DataAbertura dtreal,
	'I' tpapostilamento,
	concat(numero,'/',ano) mascmod
from
	dbo.licitacao a
join dbo.ObjetoJustificLicitacao b on a.IdObjetoJustificLicitacao = b.id
order by 3, 2
""")

for row in cur_orig:
    info_modalidades = dict_modalidades.get(row.modlic, {"Licit": "DISPENSA", "Modlic": "DI01", "Codmod": 1})

    licit = info_modalidades["Licit"]
    modlic = info_modalidades["Modlic"]
    codmod = info_modalidades["Codmod"]

    cur_dest.execute(insert, (
		row.numlic, 
		row.proclic,
		row.numlic,
		row.ano,
		row.comp,
		row.licnova,
		'S',
		row.discr[:1024],
		row.detalhe,
		row.registropreco,
		row.microempresa,
		row.numpro,
		row.discr7,
		row.datae,
		row.processo_data,
		row.dthom,
		row.dthom,
		row.codtce,
		row.ano,
		modlic,
		licit,
		codmod,
		row.dtpub,
		row.dthom,
		row.valor,
		empresa,
		row.processo,
		row.processo_ano,
		row.datae,
		None,
		None,
        row.mascmod
	))
commit()

cur_dest.execute("""UPDATE CADLIC SET FK_MODLICANO = (SELECT PK_MODLICANO FROM MODLICANO WHERE CODMOD = CADLIC.CODMOD AND ANOMOD = CADLIC.ANO AND CADLIC.EMPRESA = MODLICANO.EMPRESA) WHERE CODMOD IS NOT NULL""")
commit()

### CADLIC SESSAO

In [ ]:
cur_dest.execute("""INSERT INTO CADLIC_SESSAO (NUMLIC, SESSAO, DTREAL, HORREAL, COMP, DTENC, HORENC, SESSAOPARA, MOTIVO) 
SELECT L.NUMLIC, CAST(1 AS INTEGER), L.DTREAL, L.HORREAL, L.COMP, L.DTENC, L.HORENC, CAST('T' AS VARCHAR(1)), CAST('O' AS VARCHAR(1)) FROM CADLIC L 
WHERE numlic not in (SELECT FIRST 1 S.NUMLIC FROM CADLIC_SESSAO S WHERE S.NUMLIC = L.NUMLIC)""")
commit()

### PARTICIPANTES

In [ ]:
limpa_tabela(("prolics","prolic"))

insert_prolic = cur_dest.prep("insert into prolic(numlic,codif,nome,status) values(?,?,?,?)")
insert_prolics = cur_dest.prep("insert into prolics(sessao,numlic,codif,habilitado,status,cpf,representante) values(?,?,?,?,?,?,?)")

cur_orig.execute("""
select
	a.IdLicitacao numlic,
	b.codif,
	b.NomeRazaoSocial nome,
	a.IdSituacaoLicitante,
	b.cgccpf 
from
	dbo.Licitante a
left join (select a.Id IdFornecedorUG, b.Id codif, b.NomeRazaoSocial, CPFCNPJ cgccpf from dbo.FornecedorUG a join dbo.Pessoa b on a.IdPessoa = b.Id) b on a.IdFornecedorUG = b.IdFornecedorUG 
""")

for row in cur_orig:
    codif = fornecedores[row.codif]
    cur_dest.execute(insert_prolic, (row.numlic, codif, to_cp1252_safe(row.nome[:40]), 'A'))
    cur_dest.execute(insert_prolics, (1, row.numlic, codif, 'S', 'A', row.cgccpf, to_cp1252_safe(row.nome[:100])))
commit()

### CADPROLIC

In [ ]:
limpa_tabela(("cadprolic_detalhe_fic", "cadprolic_detalhe", "cadprolic"))
cria_coluna("cadprolic", "codant")

insert_cadprolic = cur_dest.prep("""
INSERT INTO
		cadprolic(item,
		item_mask,
		lotelic,
		numorc,
		itemorc,
		cadpro,
		quan1,
		vamed1,
		vatomed1,
		codccusto,
		reduz,
		numlic,
        microempresa,
	    item_lc147,
        codant,
		tlance)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, '$')
""")

cur_orig.execute("""
select
	row_number() over (partition by IdLicitacao order by il.IdLicitacao, IdLoteLicitacao, sequencia) item,
	format(il.IdLoteLicitacao, '00000000') lotelic,
	iditem codreduz,
	il.Quantidade quan1,
	il.ValorUnitario vamed1,
	quantidade*il.ValorUnitario vatomed1,
	il.IdLicitacao numlic,
	concat(coalesce(IdLoteLicitacao,0),'.',sequencia) codant
from
	dbo.ItemLicitacao il
""")

for row in cur_orig:
    try:
        cadpro = cadest[str(row.codreduz)]
    except KeyError:
        continue

    cur_dest.execute(insert_cadprolic, (row.item, row.item, row.lotelic, None, None, cadpro, row.quan1, row.vamed1, row.vatomed1, 0, 'N', row.numlic, 'N', None, row.codant))
commit()

cur_dest.execute("""
	INSERT INTO CADPROLIC_DETALHE (NUMLIC,item,CADPRO,quan1,VAMED1,VATOMED1,marca,CODCCUSTO,ITEM_CADPROLIC, item_lc147)
	select numlic, item, cadpro, quan1, vamed1, vatomed1, marca, codccusto, item, item_lc147 from cadprolic b where
	not exists (select 1 from cadprolic_detalhe c where b.numlic = c.numlic and b.item = c.item);
""")
commit()

cur_dest.execute("""
	insert into 
	cadprolic_detalhe_fic (numlic, item, codigo, qtd, valor, qtdadt, valoradt, codccusto, qtdmed, valormed, tipo) 
	select numlic, item, item, quan1, vatomed1, quan1, vatomed1, codccusto, quan1, vatomed1, 'C' from cadprolic b where
	not exists (select 1 from cadprolic_detalhe_fic c where b.numlic = c.numlic and b.item = c.item);
""")
commit()

cur_dest.execute("""INSERT INTO cadlotelic (descr, lotelic, numlic) SELECT distinct 'Lote ' || lotelic, lotelic, numlic FROM 
	cadprolic a WHERE lotelic IS NOT NULL AND NOT EXISTS ( SELECT 1 FROM CADLOTELIC c WHERE c.numlic = a.numlic AND a.lotelic = c.lotelic)""")
commit()

### PROPOSTA

In [ ]:
limpa_tabela(("cadpro", "cadpro_final", "cadpro_proposta"))

insert = cur_dest.prep("""
INSERT
    INTO
    cadpro_proposta(sessao,
    codif,
    item,
    itemp,
    quan1,
    vaun1,
    vato1,
    numlic,
    status,
    subem,
    marca,
    itemlance)
VALUES(?,?,?,?,?,?,?,?,?,?,?,?)""")

cur_orig.execute("""
with cte as 
(select
	1 sessao,
	codif,
	item,
	a.Quantidade quan1,
	ValorUnitario vaun1,
	a.Quantidade*ValorUnitario vato1,
	a.IdLicitacao numlic,
	'A' status,
	1 subem,
	Descricao marca,
	'S' itemlance
from
	dbo.ClassifFinalLicitacao a
join dbo.Licitante b on
	a.IdLicitante = b.Id
left join (select a.Id IdFornecedorUG, b.Id codif, b.NomeRazaoSocial, CPFCNPJ cgccpf from dbo.FornecedorUG a join dbo.Pessoa b on a.IdPessoa = b.Id) c on c.IdFornecedorUG = b.IdFornecedorUG 
join dbo.ItemDisputaLicitacao d on d.id = a.IdItemDisputaLicitacao
join (select
		row_number() over (partition by IdLicitacao order by il.IdLicitacao, IdLoteLicitacao, sequencia) item,
		il.id,
		concat(coalesce(IdLoteLicitacao,0),'.',sequencia) codant
	from
		dbo.ItemLicitacao il) e on e.id = d.IdItemLicitacao 
left join dbo.marca f on a.IdMarca = f.id 
union all
select
	1 sessao,
	codif,
	item,
	a.Quantidade quan1,
	a.ValorUnitario vaun1,
	a.Quantidade*a.ValorUnitario vato1,
	cfl.IdLicitacao numlic,
	'A' status,
	cfl.FlVencedor subem,
	Descricao marca,
	'S' itemlance
from
	dbo.DetalheClassifFinalLic  a
join dbo.ClassifFinalLicitacao cfl on a.IdClassifFinalLicitacao = cfl.id
join dbo.Licitante b on cfl.IdLicitante = b.id
left join (select a.Id IdFornecedorUG, b.Id codif, b.NomeRazaoSocial, CPFCNPJ cgccpf from dbo.FornecedorUG a join dbo.Pessoa b on a.IdPessoa = b.Id) c on c.IdFornecedorUG = b.IdFornecedorUG 
join (select
		row_number() over (partition by IdLicitacao
	order by
		il.IdLicitacao,
		IdLoteLicitacao,
		sequencia) item,
		il.id
	from
		dbo.ItemLicitacao il) d on d.id = a.IdItemLicitacao
left join dbo.marca f on a.IdMarca = f.id 
union all
select
	1 sessao,
	codif,
	item,
	a.Quantidade quan1,
	a.ValorUnitario vaun1,
	a.Quantidade*a.ValorUnitario vato1,
	a.IdLicitacao numlic,
	'A' status,
	a.FlVencedor subem,
	Descricao marca,
	'S' itemlance
from
	dbo.ClassifFinalLicitacao a
join dbo.Licitante b on a.IdLicitante = b.id
left join (select a.Id IdFornecedorUG, b.Id codif, b.NomeRazaoSocial, CPFCNPJ cgccpf from dbo.FornecedorUG a join dbo.Pessoa b on a.IdPessoa = b.Id) c on c.IdFornecedorUG = b.IdFornecedorUG 
join (select
		row_number() over (partition by IdLicitacao
	order by
		il.IdLicitacao,
		IdLoteLicitacao,
		sequencia) item,
		il.id
	from
		dbo.ItemLicitacao il) d on d.id = a.IdItemLicitacao
left join dbo.marca f on a.IdMarca = f.id)
select distinct * from cte where subem = 1
""")           

for row in cur_orig:
    try:
        codif = fornecedores[row.codif]
        cur_dest.execute(insert, (row.sessao, codif, row.item, row.item, row.quan1, row.vaun1, row.vato1, row.numlic, row.status, row.subem, row.marca, row.itemlance))
    except Exception as e:
        print(f"Error inserting row: {row.numlic}, {row.codif}, {row.item} - {e}")
commit()

cur_dest.execute("""update cadlic set comp = 3 where exists (select 1 from cadpro_proposta x where x.numlic = cadlic.numlic)""")
commit()

### CADPRO LANCE

In [ ]:
cur_dest.execute("""insert into cadpro_lance (sessao, rodada, codif, itemp, vaunl, vatol, status, subem, numlic)
	SELECT sessao, 1 rodada, CODIF, ITEMP, VAUN1, VATO1, 'F' status, SUBEM, numlic FROM CADPRO_PROPOSTA cp where subem = 1 and not exists
	(select 1 from cadpro_lance cl where cp.codif = cl.codif and cl.itemp = cp.itemp and cl.numlic = cp.numlic)""")
commit()

### CADPRO FINAL

In [ ]:
cur_dest.execute("""INSERT into cadpro_final (numlic, ult_sessao, codif, itemp, vaunf, vatof, STATUS, subem)
	SELECT numlic, sessao, codif, itemp, vaun1, vato1, CASE WHEN status = 'F' THEN 'C' ELSE status end, subem FROM cadpro_proposta
	WHERE NOT EXISTS (SELECT 1 FROM cadpro_final f WHERE f.numlic = cadpro_proposta.numlic AND f.itemp = cadpro_proposta.itemp AND f.codif = cadpro_proposta.codif)""")
commit()

cur_dest.execute("""update cadlic a SET comp = 3 WHERE comp = 1 AND EXISTS (SELECT 1 FROM cadpro_proposta x WHERE subem = 1 AND x.numlic = a.numlic) AND NOT EXISTS (SELECT 1 FROM cadpro x WHERE x.numlic = a.numlic)""")
commit()

### CADPRO_STATUS

In [ ]:
cur_dest.execute("""
    INSERT INTO cadpro_status (numlic, sessao, itemp, item, telafinal)
    SELECT b.NUMLIC, 1 AS sessao, a.item, a.item, 'I_ENCERRAMENTO'
    FROM CADPROLIC a
    JOIN cadlic b ON a.NUMLIC = b.NUMLIC
    WHERE NOT EXISTS (
        SELECT 1
        FROM cadpro_status c
        WHERE a.numlic = c.numlic
    )
    AND EXISTS (
        SELECT 1
        FROM cadlic d
        WHERE d.numlic = a.numlic AND d.comp = 3
    );
""")
commit()

### CADPRO

In [ ]:
cria_coluna("cadpro", "codant")

cur_dest.execute("""INSERT INTO CADPRO (
    CODIF,
    CADPRO,
    QUAN1,
    VAUN1,
    VATO1,
    SUBEM,
    STATUS,
    ITEM,
    NUMORC,
    ITEMORCPED,
    CODCCUSTO,
    FICHA,
    ELEMENTO,
    DESDOBRO,
    NUMLIC,
    ULT_SESSAO,
    ITEMP,
    QTDADT,
    QTDPED,
    VAUNADT,
    VATOADT,
    PERC,
    QTDSOL,
    ID_CADORC,
    VATOPED,
    VATOSOL,
    TPCONTROLE_SALDO,
    QTDPED_FORNECEDOR_ANT,
    VATOPED_FORNECEDOR_ANT,
    marca
)
SELECT
    a.CODIF,
    c.CADPRO,
    CASE WHEN a.VAUNL <> 0 THEN ROUND((a.vatol / a.VAUNL), 2) ELSE 0 END qtdunit,
    a.VAUNL,
    CASE WHEN a.VAUNL <> 0 THEN ROUND((a.vatol / a.VAUNL), 2) * a.VAUNL ELSE 0 END VATOTAL,
    1,
    'C',
    c.ITEM,
    c.NUMORC,
    c.ITEM,
    c.CODCCUSTO,
    c.FICHA,
    c.ELEMENTO,
    c.DESDOBRO,
    a.NUMLIC,
    1,
    b.ITEMP,
    CASE WHEN a.VAUNL <> 0 THEN ROUND((a.vatol / a.VAUNL), 2) ELSE 0 END qtdunit,
    0,
    a.VAUNL,
    CASE WHEN a.VAUNL <> 0 THEN ROUND((a.vatol / a.VAUNL), 2) * a.VAUNL ELSE 0 END vatoadt,
    0,
    0,
    c.ID_CADORC,
    0,
    0,
    'Q',
    0,
    0,
    p.marca
FROM
    CADPRO_LANCE a
INNER JOIN CADPRO_STATUS b ON
    b.NUMLIC = a.NUMLIC AND a.ITEMP = b.ITEMP AND a.SESSAO = b.SESSAO
INNER JOIN CADPROLIC_DETALHE c ON
    c.NUMLIC = a.NUMLIC AND b.ITEMP = c.ITEM_CADPROLIC
INNER JOIN CADLIC D ON
    D.NUMLIC = A.NUMLIC
inner join cadpro_proposta p on 
    p.numlic = a.numlic and p.itemp = a.itemp and p.codif = a.codif
WHERE
    a.SUBEM = 1 AND a.STATUS = 'F'
    AND NOT EXISTS (
        SELECT 1 
        FROM CADPRO cp
        WHERE cp.NUMLIC = a.NUMLIC 
        AND cp.ITEM = c.ITEM 
        AND cp.CODIF = a.CODIF
    )""")
commit()

cur_dest.execute("""
MERGE INTO cadpro a USING (SELECT numlic, item, codant FROM cadprolic) b
ON a.numlic = b.numlic AND a.item = b.item 
WHEN MATCHED THEN UPDATE SET a.codant = b.codant
""")
commit()

### REGPRECO

In [ ]:
cur_dest.execute("""
EXECUTE BLOCK AS  
    BEGIN  
    INSERT INTO REGPRECODOC (NUMLIC, CODATUALIZACAO, DTPRAZO, ULTIMA)  
    SELECT DISTINCT A.NUMLIC, 0, DATEADD(1 YEAR TO A.DTHOM), 'S'  
    FROM CADLIC A WHERE A.REGISTROPRECO = 'S' AND A.DTHOM IS NOT NULL  
    AND NOT EXISTS(SELECT 1 FROM REGPRECODOC X  
    WHERE X.NUMLIC = A.NUMLIC);  

    INSERT INTO REGPRECO (COD, DTPRAZO, NUMLIC, CODIF, CADPRO, CODCCUSTO, ITEM, CODATUALIZACAO, QUAN1, VAUN1, VATO1, QTDENT, SUBEM, STATUS, ULTIMA)  
    SELECT B.ITEM, DATEADD(1 YEAR TO A.DTHOM), B.NUMLIC, B.CODIF, B.CADPRO, B.CODCCUSTO, B.ITEM, 0, B.QUAN1, B.VAUN1, B.VATO1, 0, B.SUBEM, B.STATUS, 'S'  
    FROM CADLIC A INNER JOIN CADPRO B ON (A.NUMLIC = B.NUMLIC) WHERE A.REGISTROPRECO = 'S' AND A.DTHOM IS NOT NULL  
    AND NOT EXISTS(SELECT 1 FROM REGPRECO X  
    WHERE X.NUMLIC = B.NUMLIC AND X.CODIF = B.CODIF AND X.CADPRO = B.CADPRO AND X.CODCCUSTO = B.CODCCUSTO AND X.ITEM = B.ITEM);  

    INSERT INTO REGPRECOHIS (NUMLIC, CODIF, CADPRO, CODCCUSTO, ITEM, CODATUALIZACAO, QUAN1, VAUN1, VATO1, SUBEM, STATUS, MOTIVO, MARCA, NUMORC, ULTIMA)  
    SELECT B.NUMLIC, B.CODIF, B.CADPRO, B.CODCCUSTO, B.ITEM, 0, B.QUAN1, B.VAUN1, B.VATO1, B.SUBEM, B.STATUS, B.MOTIVO, B.MARCA, B.NUMORC, 'S'  
    FROM CADLIC A INNER JOIN CADPRO B ON (A.NUMLIC = B.NUMLIC) WHERE A.REGISTROPRECO = 'S' AND A.DTHOM IS NOT NULL  
    AND NOT EXISTS(SELECT 1 FROM REGPRECOHIS X  
    WHERE X.NUMLIC = B.NUMLIC AND X.CODIF = B.CODIF AND X.CADPRO = B.CADPRO AND X.CODCCUSTO = B.CODCCUSTO AND X.ITEM = B.ITEM);  
END;""")
commit()

## PEDIDOS

### ANTERIORES AO EXERCÍCIO

In [ ]:
limpa_tabela(("cadpro_saldo_ant", "regpreco_saldo_ant"))

itens_processo = {k:[v1,v2] for k,v1,v2 in cur_dest.execute("select numlic||'.'||codif||'.'||codant, cadpro, item from cadpro a JOIN cadest b USING (cadpro)").fetchall()}
itens_processo_sem_codif = {k:[v1,v2] for k,v1,v2 in cur_dest.execute("select numlic||'.'||codant, cadpro, item from cadpro a JOIN cadest b USING (cadpro)").fetchall()}

insert = cur_dest.prep("""
insert into cadpro_saldo_ant (
    ano,
    numlic,
    item,
    cadpro,
    qtdped,
    vatoped)
values (?,?,?,?,?,?)
""")

cur_orig.execute(f"""
with cte as
(select
	concat(format(Numero, '00000/'),ano%2000) numped,
	format(Numero, '00000') num,
	ano,
	DataOrdem datped,
	IdPessoa codif,
	a.id id_cadped,
	numlic,
	Finalidade obs,
	IdLocalEntregaItens,
	codant,
	coalesce(b.QuantidadeEfetiv,1) qtd,
	b.ValorUnitarioEfetiv prcunt,
	coalesce(b.QuantidadeEfetiv,1) * b.ValorUnitarioEfetiv prctot,
	IdEstrutAdministrativa codccusto,
	b.IdItemValorContrato 
from
	dbo.OrdemFornecimento a
join dbo.ItemOrdemForn b on a.id = b.IdOrdemFornecimento 
join (select
	ivc.id,
	il.IdLicitacao numlic,
	concat(coalesce(il.IdLoteLicitacao,0),'.',il.sequencia) codant
from
	dbo.ItemLicitacao il
join dbo.DetalheItemLicitacao dil on il.id = dil.IdItemLicitacao
join dbo.ItemValorCompraARP ivca on ivca.IdDetalheItemLicitacao = dil.id
join dbo.ItemValorContrato ivc on ivc.IdItemValorCompraARP = ivca.id) itemlic on itemlic.id = b.IdItemValorContrato
	where not exists (select 1 from dbo.SituacaoOrdemForn x where x.IdOrdemFornecimento = a.id and IdTipoSitOrdemForn = -6) and ano < {exercicio}
union all
select
	concat(format(Numero, '00000/'),ano%2000) numped,
	format(Numero, '00000') num,
	Ano,
	DataOrdem datped,
	IdPessoa codif,
	a.id id_cadped,
	numlic,
	Finalidade obs,
	IdLocalEntregaItens,
	codant,
	coalesce(b.QuantidadeEfetiv,1) qtd,
	b.ValorUnitarioEfetiv prcunt,
	coalesce(b.QuantidadeEfetiv,1) * b.ValorUnitarioEfetiv prctot,
	IdEstrutAdministrativa codccusto,
	b.IdItemValorContrato 
from
	dbo.OrdemFornecimento a
join dbo.ItemOrdemForn b on a.id = b.IdOrdemFornecimento 
join (select
	ivc.id,
	il.IdLicitacao numlic,
	concat(coalesce(il.IdLoteLicitacao,0),'.',il.sequencia) codant
from
	dbo.ItemValorContrato ivc
join dbo.Contrato c on
	ivc.IdContrato = c.id
join dbo.CompraARP ca on
	ca.id = c.IdCompraARP
join dbo.ItemLicitacao il on
	il.IdItem = ivc.IdItem
	and il.IdLicitacao = ca.IdLicitacao
where
	ivc.IdItemValorCompraARP is null) itemlic on itemlic.id = b.IdItemValorContrato
	where not exists (select 1 from dbo.SituacaoOrdemForn x where x.IdOrdemFornecimento = a.id and IdTipoSitOrdemForn = -6) and ano < {exercicio})
select numlic, codif, codant, sum(qtd) qtdped, sum(prctot) vatoped from cte group by numlic, codif, codant
""")

for row in cur_orig:
    codif = fornecedores[row.codif]
    try:
        key = f'{row.numlic}.{codif}.{row.codant}'
        item = itens_processo[key]
    except:
        print(f'Chave não encontrada: {key}')
        try:
            key = f'{row.numlic}.{row.codant}'
            item = itens_processo_sem_codif[key]
        except:
            print(f'Chave não encontrada sem codif: {key}')
            continue
        
    try:
        cur_dest.execute(insert, (
			exercicio,
			row.numlic,
			item[1],
			item[0],
			row.qtdped,
			row.vatoped
		))
        commit()
    except fdb.DatabaseError as e:
        if e.args[1] == -803:
            cur_dest.execute("update cadpro_saldo_ant set qtdped = qtdped + ?, vatoped = vatoped + ? where numlic = ? and item = ?", (row.qtdped, row.vatoped, row.numlic, item[1]))
            commit()
        else:
            raise

### DO EXERCÍCIO

In [ ]:
limpa_tabela(("icadped", "cadped"))

itens_processo = {k:[v1,v2] for k,v1,v2 in cur_dest.execute("select numlic||'.'||codif||'.'||codant, cadpro, item from cadpro").fetchall()}
itens_processo_sem_codif = {k:[v1,v2] for k,v1,v2 in cur_dest.execute("select numlic||'.'||codant, cadpro, item from cadpro a").fetchall()}

inser_cadped = cur_dest.prep("insert into cadped (numped, num, ano, datped, codif, id_cadped, empresa, numlic, obs, codccusto) values (?,?,?,?,?,?,?,?,?,?)")
insert_icadped = cur_dest.prep("insert into icadped (numped, item, cadpro, qtd, prcunt, prctot, codccusto, id_cadped) values (?,?,?,?,?,?,?,?)")

cur_orig.execute(f"""
select
	concat(format(Numero, '00000/'),ano%2000) numped,
	format(Numero, '00000') num,
	ano,
	DataOrdem datped,
	IdPessoa codif,
	a.id id_cadped,
	numlic,
	Finalidade obs,
	IdLocalEntregaItens,
	codant,
	coalesce(b.QuantidadeEfetiv,1) qtd,
	b.ValorUnitarioEfetiv prcunt,
	coalesce(b.QuantidadeEfetiv,1) * b.ValorUnitarioEfetiv prctot,
	IdEstrutAdministrativa codccusto,
	b.IdItemValorContrato 
from
	dbo.OrdemFornecimento a
join dbo.ItemOrdemForn b on a.id = b.IdOrdemFornecimento 
join (select
	ivc.id,
	il.IdLicitacao numlic,
	concat(coalesce(il.IdLoteLicitacao,0),'.',il.sequencia) codant
from
	dbo.ItemLicitacao il
join dbo.DetalheItemLicitacao dil on il.id = dil.IdItemLicitacao
join dbo.ItemValorCompraARP ivca on ivca.IdDetalheItemLicitacao = dil.id
join dbo.ItemValorContrato ivc on ivc.IdItemValorCompraARP = ivca.id) itemlic on itemlic.id = b.IdItemValorContrato
	where not exists (select 1 from dbo.SituacaoOrdemForn x where x.IdOrdemFornecimento = a.id and IdTipoSitOrdemForn = -6) and ano = {exercicio}
union all
select
	concat(format(Numero, '00000/'),ano%2000) numped,
	format(Numero, '00000') num,
	Ano,
	DataOrdem datped,
	IdPessoa codif,
	a.id id_cadped,
	numlic,
	Finalidade obs,
	IdLocalEntregaItens,
	codant,
	coalesce(b.QuantidadeEfetiv,1) qtd,
	b.ValorUnitarioEfetiv prcunt,
	coalesce(b.QuantidadeEfetiv,1) * b.ValorUnitarioEfetiv prctot,
	IdEstrutAdministrativa codccusto,
	b.IdItemValorContrato 
from
	dbo.OrdemFornecimento a
join dbo.ItemOrdemForn b on a.id = b.IdOrdemFornecimento 
join (select
	ivc.id,
	il.IdLicitacao numlic,
	concat(coalesce(il.IdLoteLicitacao,0),'.',il.sequencia) codant
from
	dbo.ItemValorContrato ivc
join dbo.Contrato c on
	ivc.IdContrato = c.id
join dbo.CompraARP ca on
	ca.id = c.IdCompraARP
join dbo.ItemLicitacao il on
	il.IdItem = ivc.IdItem
	and il.IdLicitacao = ca.IdLicitacao
where
	ivc.IdItemValorCompraARP is null) itemlic on itemlic.id = b.IdItemValorContrato
	where not exists (select 1 from dbo.SituacaoOrdemForn x where x.IdOrdemFornecimento = a.id and IdTipoSitOrdemForn = -6) and ano = {exercicio}
order by id_cadped
""")

id_cadped = 0

for row in cur_orig:
    codif = fornecedores[row.codif]
    if row.id_cadped != id_cadped:
        id_cadped = row.id_cadped
        cur_dest.execute(inser_cadped, (
            row.numped,
            row.num,
            row.ano,
            row.datped,
            codif,
            id_cadped,
            empresa,
            row.numlic,
            row.obs,
            row.codccusto
        ))
        commit()
    
    try:
        key = f'{row.numlic}.{codif}.{row.codant}'
        item = itens_processo[key]
    except:
        print(f'Chave não encontrada: {key} - pedido: {row.num}')
        try:
            key = f'{row.numlic}.{row.codant}'
            item = itens_processo_sem_codif[key]
        except:
            print(f'Chave não encontrada sem codif: {key} - pedido: {row.num}')
            continue

    try:
        cur_dest.execute(insert_icadped, (
            row.numped,
            item[1],
            item[0],
            row.qtd,
            row.prcunt,
            row.prctot,
            row.codccusto,
            id_cadped
        ))
    except:
        print(f'Erro ao inserir pedido {row.num}')
        continue
commit()

cur_dest.execute("""
UPDATE cadped a SET a.codatualizacao_rp = (SELECT max(codatualizacao) 
FROM regprecodoc x WHERE x.numlic = a.numlic)
""")
commit()

cur_dest.execute("""
EXECUTE BLOCK
AS
DECLARE VARIABLE NUMLIC INTEGER;
DECLARE VARIABLE ITEM INTEGER;
DECLARE VARIABLE CODCCUSTO INTEGER;
BEGIN
  FOR
      WITH freq AS (
          SELECT
              numlic,
              a.item,
              a.codccusto,
              COUNT(*) AS qtd,
              ROW_NUMBER() OVER (
                  PARTITION BY numlic, a.item
                  ORDER BY COUNT(*) DESC, a.codccusto
              ) AS rn
          FROM icadped a
          JOIN cadped c USING (id_cadped)
          GROUP BY numlic, a.item, a.codccusto
      )
      SELECT
          numlic,
          item,
          codccusto
      FROM freq
      WHERE rn = 1
      INTO :NUMLIC, :ITEM, :CODCCUSTO
  DO BEGIN
      UPDATE CADPROLIC p
         SET p.CODCCUSTO = :CODCCUSTO
       WHERE p.ITEM = :ITEM
         AND p.NUMLIC = :NUMLIC;

      UPDATE CADPROLIC_DETALHE d
         SET d.CODCCUSTO = :CODCCUSTO
       WHERE d.ITEM = :ITEM
         AND d.NUMLIC = :NUMLIC;

      UPDATE CADPROLIC_DETALHE_FIC f
         SET f.CODCCUSTO = :CODCCUSTO
       WHERE f.ITEM = :ITEM
         AND f.NUMLIC = :NUMLIC;

      UPDATE CADPRO c
         SET c.CODCCUSTO = :CODCCUSTO
       WHERE c.ITEM = :ITEM
         AND c.NUMLIC = :NUMLIC;

      UPDATE REGPRECO r
         SET r.CODCCUSTO = :CODCCUSTO
       WHERE r.ITEM = :ITEM
         AND r.NUMLIC = :NUMLIC;

      UPDATE REGPRECOHIS h
         SET h.CODCCUSTO = :CODCCUSTO
       WHERE h.ITEM = :ITEM
         AND h.NUMLIC = :NUMLIC;
  END
END
""")
commit()

# 📄 CONTRATOS

## CADASTROS

In [ ]:
limpa_tabela(('contratos',))

insert = cur_dest.prep("""
INSERT
	INTO
	contratos (codigo,
	contratonum,
	empresa,
	ano,
	dtassi,
	vigeni,
	vigenf,
	dtpubl,
	veicpub,
	nproli,
	flegal,
	fundlegal,
	codif,
	objeti,
	objeto,
	objeto_completo,
	garant,
	valcon,
	tipo_contrato,
	ataregpreco,
	tipoco,
	proclic) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

cur_orig.execute("""
select distinct
	concat(format(a.Id, '0000/'),a.ano%2000) codigo,
	concat(format(a.Numero, '0000/'),a.ano%2000) contratonum,
	a.ano ano,
	cast(DataAssinatura as date) dtassi,
	cast(DataInicioVigencia as date) vigeni,
	cast(DataFimVigencia as date) vigenf,	
	cast(DataAssinatura as date) dtpubl,
	format(l.id , '000000') nproli,
	codif,
	DescricaoObjeto objeto,
	ValorContratadoCompraARP valcon,
	concat(format(l.id , '000000/'), l.ano%2000) proclic
from
	dbo.contrato a
join (select
	IdContrato,
	max(IdDadosGeraisContrato) IdDados,
	max(IdDadosFinanceirosContrato) idFinanceiro,
	max(pc.IdPessoa) codif
from
	dbo.ContratoComposicao cc
left join dbo.ParteContrato pc on cc.IdParteContrato = pc.id and abs(pc.IdTipoParteContrato) = 1
where
	FlAtual <> 0
group by
	IdContrato) b on a.id = b.IdContrato
left join dbo.DadosGeraisContrato c on c.Id = b.IdDados  
left join dbo.compraARP d on a.IdCompraARP = d.id
join dbo.Licitacao l on l.id = d.IdLicitacao
left join dbo.DadosFinanceirosContrato dfc on dfc.id = b.idFinanceiro 
order by 1
""")

for row in cur_orig:
    codif = fornecedores[row.codif]
    cur_dest.execute(insert, (row.codigo, row.contratonum, empresa, row.ano, row.dtassi, row.vigeni, row.vigenf, row.dtpubl, None, row.nproli, 'LICITAÇÃO', 'LICITAÇÃO', codif, to_cp1252_safe(row.objeto[:60]), to_cp1252_safe(row.objeto[:60]), to_cp1252_safe(row.objeto), 'SG', row.valcon, 'P', 'N', '01', row.proclic))
commit()

cur_dest.execute("""
MERGE INTO contratos a USING (SELECT proclic, licit, registropreco, numpro FROM cadlic) b
ON a.proclic = b.proclic
WHEN MATCHED THEN UPDATE SET a.MODALI = substring(b.licit from 1 FOR 20), ataregpreco = registropreco, numlicmod = numpro
""") 
commit()

cur_dest.execute("""
merge into contratos a using (select codigo, sum(vadem-anula) empenhado from despes a
join (select codigo, cast(substring(codigo from 1 for 4) as integer) id from contratos)
on a.id_contrato_ant = id
group by 1) b on a.codigo = b.codigo
when matched then update set a.empenhado = b.empenhado
""") 
commit()

## ADITAMENTOS

In [ ]:
select
	concat(format(c.Id, '0000/'),c.ano%2000) contrato,
	cast(DataEfetivacao as date) dtlan,
	Descricao,
	concat(format(ac.Numero, '0000/'),year(dataEfetivacao)%2000) termo,
	ac.numero codigo,
	ac.id
from
	AditivoContrato ac
join Contrato c on ac.IdContrato = c.id
where ac.DataCancelamento is null
order by ac.IdContrato, ac.DataEfetivacao, ac.Sequencia 